In [1]:
import os
from dotenv import load_dotenv
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

load_dotenv()

True

In [2]:
# template = """Question: {question}
# 
# Answer: Let's think step by step."""
# 
# prompt = PromptTemplate(template=template, input_variables=["question"])

In [3]:
llm = OpenAI(openai_api_key=os.getenv('OPENAI_API_KEY'))
# llm_chain = LLMChain(prompt=prompt, llm=llm)

In [4]:
# question = "What NFL team won the Super Bowl in the year Justin Beiber was born?"
# 
# llm_chain.run(question)

In [5]:
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate

system_template = """Create an informative and comprehensive answer for a given question based solely on the given documents. You must only use information from the given documents.
Use an unbiased and journalistic tone. Do not repeat text.
Cite the documents using [Document name] notation.
If multiple documents contain the answer, cite those documents like ‘as stated in [Document name 1], [Document name 2], etc.’.
You must include citations in your answer.
If the documents do not contain the answer to the question, say that  ‘answering is not possible given the available information.’
{context}

"""
messages = [
        SystemMessagePromptTemplate.from_template(system_template),
        HumanMessagePromptTemplate.from_template("{question}"),
    ]
prompt = ChatPromptTemplate.from_messages(messages)

In [6]:
from langchain.embeddings import OpenAIEmbeddings

faiss_local_path = '../data/embedded_dataset/faiss/openai_1000/faiss_idx'
embeddings = OpenAIEmbeddings(openai_api_key=os.getenv('OPENAI_API_KEY'))

In [7]:
from langchain import FAISS

db = FAISS.load_local(faiss_local_path, embeddings)

In [11]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

conversation_chain = ConversationalRetrievalChain.from_llm(
        llm,
        db.as_retriever(search_kwargs={"k": 3}),
        memory=ConversationBufferMemory(
        memory_key="chat_history", return_messages=True, output_key="answer"
    ),
        return_source_documents=True,
        combine_docs_chain_kwargs={
            "document_prompt": PromptTemplate(
                input_variables=["page_content", "Source"],
                template="Document name: {Source}\nContext:\n{page_content}",
            ),
            "prompt": prompt,
        },
    )

In [24]:
response = conversation_chain('What are the problems with the military standard IP specification?')

In [25]:
print(response['answer'])



Answer: According to RFC 963, some issues associated with the military standard Internet Protocol (MIL-STD-1777) specification include the lack of a mechanism for reliability, flow control, sequencing, etc. as it provides only a pure datagram service. As stated in RFC 1500, the Internet community and DoD MIL-STD specifications for IP and TCP are intended to describe the same protocols, but they differ in style and level of detail. Additionally, the Internet and DoD MIL-STD specifications for FTP, SMTP, and Telnet protocols are essentially the same documents. Therefore, any differences between the protocols specified by these sets of documents should be reported to DISA and the IESG. [RFC 963], [RFC 1500]
